# Class Exercises - Week 6

---
---

**Author:** Dr Giordano Scarciotti (g.scarciotti@imperial.ac.uk) - Imperial College London

**Module:** ELEC70066 - Applied Advanced Optimisation

**Version:** 1.1.1 - 13/02/2025

---
---

**Reminder:** The solutions of some these exercises may be found online. Do not look for the solutions, but ask for help to me or the GTA. Your assessment will not have a solution available online so it is of paramount importance that you learn how to solve the problems without relying on a solution being available.

The solutions of these exercises will be discussed at the end of the session. Moreover, the complete solution will be provided to you after 1 week.  **N.B.** **Do not share the solutions online**

## Exercise 1

Flux balance analysis is based on a very simple model of the reactions going on in a cell, keeping track only of the gross rate of consumption and production of various chemical species within the cell. Based on the known [stoichiometry](https://en.wikipedia.org/wiki/Stoichiometry) of the reactions, and known upper bounds on some of the reaction rates, we can compute bounds on the other reaction rates, or cell growth, for example.

We focus on $m$ metabolites in a cell, labeled $M_1$, ..., $M_m$. There are $n$ reactions going on, labeled $R_1$, ..., $R_n$, with nonnegative reaction rates $v_1$, ..., $v_n$. Each reaction has a (known) stoichiometry, which tells us the rate of consumption and production of the metabolites per unit of reaction rate. The stoichiometry data is given by the stoichiometry matrix $S \in \mathbb{R}^{m\times n}$, defined as follows: $S_{ij}$ is the rate of production of $M_i$ due to unit reaction rate $v_j = 1$. Here we consider consumption of a metabolite as negative production; so for example $S_{ij}=-2$ means that reaction $R_j$ causes metabolite $M_i$ to be consumed at a rate $2v_j$.

As an example, suppose reaction $R_1$ has the form $M_1 \to M_2 + 2M_3$. The consumption rate of $M_1$, due to this reaction, is $v_1$; the production rate of $M_2$ is $v_1$; and the production rate of $M_3$ is $2v_1$. The reaction $R_1$ has no effect on metabolites $M_4$, ..., $M_m$. This corresponds to a first column of $S$ of the form $(-1,1,2,0,\dots,0)$.

Reactions are also used to model flow of metabolites into and out of the cell. For example, suppose that reaction $R_2$ corresponds to the flow of metabolite $M_1$ into the cell, with $v_2$ giving the flow rate. This corresponds to a second column of $S$ of the form $(1,0,\dots,0)$.

The last reaction, $R_n$, corresponds to biomass creation, or cell growth, so the reaction rate $v_n$ is the cell growth rate. The last column of $S$ gives the amounts of metabolites used or created per unit of cell growth rate.

Since our reactions include metabolites entering or leaving the cell, as well as those converted to biomass within the cell, we have conservation of the metabolites, which can be expressed as $Sv=0$. In addition, we are given upper limits on some of the reaction rates, which we express as $v \preccurlyeq v^{\max}$. The goal is to find the maximum possible cell growth rate consistent with the constraints.



Consider the data

In [1]:
import cvxpy as cp
import numpy as np
S = np.array([[1,	0,	0,	0,	0,	0],
              [-1,	1,	0,	0,	0,	0],
              [-1,	0,	1,	0,	0,	0],
              [0,	-1,	0,	2,	-1,	0],
              [0,	0,	0,	0,	1,	0],
              [0,	-2,	1,	0,	0,	1],
              [0,	0,	-1,	1,	0,	0],
              [0,	0,	0,	0,	0,	-1],
              [0,	0,	0,	-1,	0,	0]]).T

vmax = np.array([10.10,100,5.90,100,3.70,100,100,100,100])

*    What is the maximum possible cell growth rate $G^*$?

*    Which of the reaction rate limits have nonzero optimal Lagrange multipliers? You can find how to obtain the dual variables [here](https://www.cvxpy.org/tutorial/advanced/index.html#dual-variables).


*    You will see in the next lecture that the nonzero optimal Lagrange multipliers indicate the constraints towards which the optimal solution is sensitive. The larger the value of the optimal Lagrange multiplier, the larger is the deterioration of the optimal value if the corresponding constraint is made more strict. Which of the reaction rate limits is the maximum growth rate most sensitive to?

In [27]:
# Max cell growth rate
v = cp.Variable(S.shape[1])
constraints = [
    S @ v == 0, 
    v <= vmax, 
    v >= 0
]
G = v[-1]
objective = cp.Maximize(G)

problem = cp.Problem(objective, constraints)
problem.solve()
print("Optimal value: ", problem.value)

# dual variable
print(constraints[1].dual_value)

# most sensitive reaction rate limit
sensitive_reaction = np.argmax(constraints[1].dual_value)
print("Most sensitive reaction rate limit", constraints[1].dual_value[sensitive_reaction])


Optimal value:  13.549999998955114
[5.00000001e-01 6.85525515e-12 5.00000000e-01 8.10492365e-12
 1.50000000e+00 7.19017921e-12 7.34455009e-12 6.96928268e-12
 7.16996997e-12]
Most sensitive reaction rate limit 1.500000000034336


# Exercise 2

This exercise is a continuation of the previous.

For simplicity, we'll assume that each reaction is controlled by an associated gene, i.e., gene $G_i$ controls reaction $R_i$. Knocking out a set of genes associated with some reactions has the effect of setting the reaction rates (or equivalently, the associated $v^{\max}$ entries) to zero, which of course reduces the maximum possible growth rate. If the maximum growth rate becomes small enough or zero, it is reasonable to guess that knocking out the set of genes will kill the cell. An *essential gene* is one that when knocked out reduces the maximum growth rate below a given threshold $G^{\min}$. (Note that $G_n$ is always an essential gene.) A *synthetic lethal* is a pair of non-essential genes that when knocked out reduces the maximum growth rate below the threshold.

*    Given the threshold $G^{\min} = 0.2G^*$, find all essential genes.
*    Given the threshold $G^{\min} = 0.2G^*$, find all the gene pairs which are synthetic lethals.

In [ ]:
v = cp.Variable(S.shape[1])
constraints = [
    S @ v == 0, 
    v <= vmax, 
    v >= 0
]
G = v[-1]
objective = cp.Maximize(G)

problem = cp.Problem(objective, constraints)
problem.solve()
print("Optimal value: ", problem.value)

Gmin = 0.2 * v.value[-1]
print(Gmin)

# essential genes
for i in range(S.shape[0]):
    v = cp.Variable(S.shape[1])
    constraints = [
        S @ v == 0, 
        v <= vmax, 
        v >= 0
    ]
    G = v[i]
    objective = cp.Maximize(G)
    problem = cp.Problem(objective, constraints)
    problem.solve()
    if v[i].value <= Gmin:
        print(i)

# gene pairs which are synthetic lethals
gene_pairs = []
for i in range (S.shape[0]):
    for j in range(i+1, S.shape[0]):
        v = cp.Variable(S.shape[1])
        constraints = [
            S @ v == 0, 
            v <= vmax, 
            v >= 0
        ]
        G = v[-1]
        objective = cp.Maximize(G)
        problem = cp.Problem(objective, constraints)
        problem.solve()
        if v[i].value <= Gmin and v[j].value <= Gmin:
            gene_pairs.append((i,j))

print("Gene pairs which are synthetic lethals", gene_pairs)
 

Optimal value:  13.549999998955114
2.709999999791023
Gene pairs which are synthetic lethals [(0, 1), (0, 2), (3, 4), (3, 5), (4, 5)]


## Exercise 3

Formulate the following problems as LPs. Explain in detail the relation between the optimal solution of each problem and the solution of its equivalent LP.
1.   Minimize $||Ax - b||_\infty$ ($\ell_\infty$-norm approximation).
2.   Minimize $||Ax - b||_1$ ($\ell_1$-norm approximation).
3.   Minimize $||Ax - b||_1$ subject to $||x||_\infty \le 1$.
4.   Minimize $||x||_1$ subject to $||Ax - b||_\infty \le 1$.
5.   Minimize $||Ax - b||_1 + ||x||_\infty$.

In each problem, $A \in \mathbb{R}^{m\times n}$ and $b\in  \mathbb{R}^{m}$ are given.